## Scenario 1: Local to Remote with MLFlow (Tracking)

1)	User trains locally using MLFlow using MLFlow logging / autologging

2)	Move to AML by setting the tracking URI in the backend (not in my training code), packaging as a project and using the AzureML CLIv2 or MLFlow CLI. The user doesn’t have to change their training or scoring code.

    a.	Help users set MLFLOW_TRACKING_URI via CLI (can we get TRACKING URI from the CLI?)

        i.	Use: az ml workspace show | grep "mlflow_tracking_uri"

    b.	User should be able to run same code

        i.	python train.py

    c.	The user can run that same training script locally and still have everything tracked in AML without updating code

3)	All AML parameters, metrics, models and logged artifacts should show up in AML run history

**NOTE** This only works if the user has set an experiment in their code, if not it break. We need to be able to set a default experiment for the user with now experiment set.

Doc: https://microsoft-my.sharepoint.com/:w:/p/osomorog/Ed7l1SLKac9Irz_PY-XnXaQB90-WeAosazcQOT24PRd3-g?e=hKS09V 

## User trains locally using MLFlow using MLFlow logging / autologging

In [2]:
# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license.

import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.sklearn

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

with mlflow.start_run():
    X, y = load_diabetes(return_X_y=True)
    columns = ['age', 'gender', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    data = {
        "train": {"X": X_train, "y": y_train},
        "test": {"X": X_test, "y": y_test}}

    mlflow.log_metric("Training samples", len(data['train']['X']))
    mlflow.log_metric("Test samples", len(data['test']['X']))

    # Log the algorithm parameter alpha to the run
    mlflow.log_metric('alpha', 0.03)
    # Create, fit, and test the scikit-learn Ridge regression model
    regression_model = Ridge(alpha=0.03)
    regression_model.fit(data['train']['X'], data['train']['y'])
    preds = regression_model.predict(data['test']['X'])

    # Log mean squared error
    print('Mean Squared Error is', mean_squared_error(data['test']['y'], preds))
    mlflow.log_metric('mse', mean_squared_error(data['test']['y'], preds))

    # Save the model to the outputs directory for capture
    mlflow.sklearn.log_model(regression_model, "model")

    # Plot actuals vs predictions and save the plot within the run
    fig = plt.figure(1)
    idx = np.argsort(data['test']['y'])
    plt.plot(data['test']['y'][idx], preds[idx])
    fig.savefig("actuals_vs_predictions.png")
    mlflow.log_artifact("actuals_vs_predictions.png")

Mean Squared Error is 3424.900315896017


Look for the **/mlruns** folder in your current directory to find the run details

## Move to AML by setting the tracking URI in the backend (not in my training code), and using MLFlow CLI. 

**NOTE:** The Compute instance team will be added in Tracking_URI to every COmpute Instance in the future

In [1]:
!az extension add -n ml -y 

Extension 'ml' is already installed.


In [4]:
!az login

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AZGDYBZ96 to authenticate.
The following tenants don't contain accessible subscriptions. Use 'az login --allow-no-subscriptions' to have tenant level access.
b1c14d5c-3625-45b3-a430-9552373a0c2f 'SAS'
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "d6f18478-178f-450d-8f3e-a891b4d83dd0",
    "id": "59ea72cc-1f3e-4d29-bd60-242d81b57b05",
    "isDefault": false,
    "managedByTenants": [],
    "name": "Visual Studio Enterprise Subscription",
    "state": "Enabled",
    "tenantId": "d6f18478-178f-450d-8f3e-a891b4d83dd0",
    "user": {
      "name": "osomorog@microsoft.com",
      "type": "user"
    }
  },
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "72f988bf-86f1-41af-91ab-2d7cd011db47",
    "id": "316a14b3-f94a-4267-a76a-497e589ef0a8",
    "isDefault": false,
    "managedByTenants": [],
    "name": "Ranker training",
    "state": "Enabled",
    "tenantId": "72f988bf-86f

In [2]:
import subprocess

#Get MLFLow UI through the Azure ML CLI v2 and convert to string
MLFLOW_TRACKING_URI = subprocess.run(["az", "ml", "workspace", "show", "--query", "mlflow_tracking_uri", "-o", "tsv"], stdout=subprocess.PIPE, text=True)
MLFLOW_TRACKING_URI = str(MLFLOW_TRACKING_URI.stdout).strip()

## Make sure the MLFLow URI looks something like this: 
## azureml://westus.api.azureml.ms/mlflow/v1.0/subscriptions/<Sub-ID>/resourceGroups/<RG>/providers/Microsoft.MachineLearningServices/workspaces/<WS>
print("MLFlow Tracking URI:", MLFLOW_TRACKING_URI)

MLFlow Tracking URI: azureml://westus.api.azureml.ms/mlflow/v1.0/subscriptions/95a911b6-47f7-4a8b-be9b-c1c2bf56579b/resourceGroups/osomorog/providers/Microsoft.MachineLearningServices/workspaces/mlflowworkspace


In [3]:
## Set the MLFLOW TRACKING URI
import mlflow
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

# Run again with AzureML as Tracking Server

**NOTE**: This only works if the user has set an experiment in their code, if not it break. We need to be able to set a default experiment for the user with now experiment set. A default experiment flow will be avaible in future release

In [4]:
# Copyright (c) Microsoft. All rights reserved.
# Licensed under the MIT license.

import numpy as np
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import mlflow
import mlflow.sklearn

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

## User currently needs to add an experiment name. Should not be neccesary in a future relase
mlflow.set_experiment("test-abe")
########

with mlflow.start_run():
    X, y = load_diabetes(return_X_y=True)
    columns = ['age', 'gender', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
    data = {
        "train": {"X": X_train, "y": y_train},
        "test": {"X": X_test, "y": y_test}}

    mlflow.log_metric("Training samples", len(data['train']['X']))
    mlflow.log_metric("Test samples", len(data['test']['X']))

    # Log the algorithm parameter alpha to the run
    mlflow.log_metric('alpha', 0.03)
    # Create, fit, and test the scikit-learn Ridge regression model
    regression_model = Ridge(alpha=0.03)
    regression_model.fit(data['train']['X'], data['train']['y'])
    preds = regression_model.predict(data['test']['X'])

    # Log mean squared error
    print('Mean Squared Error is', mean_squared_error(data['test']['y'], preds))
    mlflow.log_metric('mse', mean_squared_error(data['test']['y'], preds))

    # Save the model to the outputs directory for capture
    mlflow.sklearn.log_model(regression_model, "model")

    # Plot actuals vs predictions and save the plot within the run
    fig = plt.figure(1)
    idx = np.argsort(data['test']['y'])
    plt.plot(data['test']['y'][idx], preds[idx])
    fig.savefig("actuals_vs_predictions.png")
    mlflow.log_artifact("actuals_vs_predictions.png")

Mean Squared Error is 3424.900315896017


Check the Experiments page in AML to find the run details